In [67]:
import itertools
import nltk
import numpy as np
import seaborn as sns


In [2]:
import json
jobs = json.load(open("/Users/kalyani/01ColumbiaQMSS/01Semester2/NLP/Homework_2/indeed_title_job_desc(1).json"))




In [81]:
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import TreebankWordTokenizer, ToktokTokenizer, TweetTokenizer
from collections import Counter
from itertools import product
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

tokenizers = {'tree': TreebankWordTokenizer(),
              'toktok': ToktokTokenizer()}

stemmers = {'porter': PorterStemmer(),
            'stemmer': SnowballStemmer('english')}

word_counts = {}
word_counts_titles = {}

def create_key(tokenizer, stemmer):
    return str(tokenizer) + ":" + str(stemmer)
    
for tokenizer, stemmer in product(tokenizers.keys(), stemmers.keys()):
    word_counts[create_key(tokenizer, stemmer)] = []

count = 0
df = pd.DataFrame(columns = ["word", "count"])    
# Loop over the functions and sentences
for tokenizer, stemmer, job_title in product(tokenizers,
                                            stemmers, jobs.keys()):
    key_title = create_key(tokenizer, stemmer) + ", " + job_title
    word_counts_titles[key_title] = []

    for desc in jobs[job_title]:
        toks = tokenizers[tokenizer].tokenize(desc)
        stems = [stemmers[stemmer].stem(tok) for tok in toks]
        words = Counter(stems)
        mat = CountVectorizer(words)
        key = create_key(tokenizer, stemmer) 
        word_counts[key].append(dict(mat.input))
        word_counts_titles[key_title].append(dict(mat.input))
    df1 = pd.DataFrame(word_counts_titles[key_title])
    key_title = key_title.split(', ')
    title = key_title[1]
    tokenizer_stemmer = key_title[0]
    df1['title'] =  [str(title)] * len(df1)
    df1['tokenizer_stemmer'] = [str(tokenizer_stemmer)] * len(df1)
    df = pd.concat([df, df1], sort=True)
    
    count += 1
    
    
        
       
    
    


In [5]:
df.head()

,!,"""",#,#1,#datavisu,#ind123,#li,#li-bf,#li-bj1,#li-ct1,...,“up,“valu,“voic,“volans-i,“what-if,“x,”,•,…,﻿we
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
for key in word_counts.keys():
    print(key)
    print(len(list(word_counts[key])))

tree:porter
735
tree:stemmer
735
toktok:porter
735
toktok:stemmer
735


The dimension of the word count matrix for the four pre-processing specifications are: 735.

In [83]:
def highest_n_numbers(list1, n):
    return sorted(range(len(list1)), key=lambda i: list1[i])[-n:]

In [85]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
logreg_dict = {}
for tokenizer_stemmer in np.unique(df['tokenizer_stemmer']):
    df1 = df.loc[df['tokenizer_stemmer'] == tokenizer_stemmer]
    X = df1.loc[:, df1.columns != 'title']
    X = X.loc[:, X.columns != 'tokenizer_stemmer']
    X = X.fillna(0)
    y = df1['title'].astype('category')
    y1 = pd.DataFrame(y)
    #y1 = pd.Series(LabelEncoder().fit_transform(y))
    logreg = LogisticRegression(C=1e90).fit(X, y1)
    coefs = list(logreg.coef_)
    sums = []
    index = 0
    while index < len(coefs[0]):
        sum = coefs[0][index]+ coefs[1][index] +coefs[2][index] +coefs[3][index] +coefs[4][index]
        sums.append(sum)
        index += 1
    logreg_dict[tokenizer_stemmer] = highest_n_numbers(sums, 10)
logreg = pd.DataFrame(logreg_dict)

logreg_dict_words = {}

meep = []

for key in logreg_dict.keys():
    logreg_dict_words[key] = []
    for index in logreg_dict['toktok:porter']:
        logreg_dict_words[key].append(df.columns[index])
    

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A c

### Below are the words with the top ten coefficients for each kind of pre-processing specification

In [86]:
logreg_dict_words

{'toktok:porter': ['divers',
  'scalabl',
  'We',
  'an',
  'better',
  'workstreams.',
  'employ',
  'team.',
  ';',
  'solut'],
 'toktok:stemmer': ['divers',
  'scalabl',
  'We',
  'an',
  'better',
  'workstreams.',
  'employ',
  'team.',
  ';',
  'solut'],
 'tree:porter': ['divers',
  'scalabl',
  'We',
  'an',
  'better',
  'workstreams.',
  'employ',
  'team.',
  ';',
  'solut'],
 'tree:stemmer': ['divers',
  'scalabl',
  'We',
  'an',
  'better',
  'workstreams.',
  'employ',
  'team.',
  ';',
  'solut']}

In [94]:
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import TreebankWordTokenizer, ToktokTokenizer, TweetTokenizer
from collections import Counter
from itertools import product
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tokenizers = {'tree': TreebankWordTokenizer(),
              'toktok': ToktokTokenizer()}

stemmers = {'porter': PorterStemmer(),
            'stemmer': SnowballStemmer('english')}

word_counts = {}
word_counts_titles = {}

def create_key(tokenizer, stemmer):
    return str(tokenizer) + ":" + str(stemmer)
    
for tokenizer, stemmer in product(tokenizers.keys(), stemmers.keys()):
    word_counts[create_key(tokenizer, stemmer)] = []

count = 0
df = pd.DataFrame(columns = ["word", "count"])    
# Loop over the functions and sentences
for tokenizer, stemmer, job_title in product(tokenizers,
                                            stemmers, jobs.keys()):
    key_title = create_key(tokenizer, stemmer) + ", " + job_title
    word_counts_titles[key_title] = []

    for desc in jobs[job_title]:
        toks = tokenizers[tokenizer].tokenize(desc)
        stems = [stemmers[stemmer].stem(tok) for tok in toks]
        words = Counter(stems)
        tvectorizer = TfidfVectorizer(words, ngram_range=(1, 3))
        key = create_key(tokenizer, stemmer) 
        word_counts[key].append(dict(mat.input))
        word_counts_titles[key_title].append(dict(mat.input))
    df1 = pd.DataFrame(word_counts_titles[key_title])
    key_title = key_title.split(', ')
    title = key_title[1]
    tokenizer_stemmer = key_title[0]
    df1['title'] =  [str(title)] * len(df1)
    df1['tokenizer_stemmer'] = [str(tokenizer_stemmer)] * len(df1)
    df = pd.concat([df, df1])
    
    count += 1

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
logreg_dict = {}
for tokenizer_stemmer in np.unique(df['tokenizer_stemmer']):
    df1 = df.loc[df['tokenizer_stemmer'] == tokenizer_stemmer]
    X = df1.loc[:, df1.columns != 'title']
    X = X.loc[:, X.columns != 'tokenizer_stemmer']
    X = X.fillna(0)
    print(X.head())
    y = df1['title'].astype('category')
    y1 = pd.DataFrame(y)
    #y1 = pd.Series(LabelEncoder().fit_transform(y))
    logreg = LogisticRegression(C=1e90).fit(X, y1)
    coefs = list(logreg.coef_)
    sums = []
    index = 0
    while index < len(coefs[0]):
        #maxim = coefs[0][index]+ coefs[1][index] +coefs[2][index] +coefs[3][index] +coefs[4][index]
        maxim = max(coefs[0][index],coefs[1][index],coefs[2][index],coefs[3][index],coefs[4][index])
        sums.append(maxim)
        index += 1
        logreg_dict[tokenizer_stemmer] = highest_n_numbers(sums, 10)
logreg = pd.DataFrame(logreg_dict)

logreg_dict_words = {}

meep = []

for key in logreg_dict.keys():
    logreg_dict_words[key] = []
    for index in logreg_dict['toktok:porter']:
        logreg_dict_words[key].append(df.columns[index])
    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



     %    '    (    )     ,    -    .    2    3    5 ...   without  word  \
0  1.0  6.0  4.0  4.0  47.0  1.0  1.0  3.0  1.0  3.0 ...       1.0     0   
1  1.0  6.0  4.0  4.0  47.0  1.0  1.0  3.0  1.0  3.0 ...       1.0     0   
2  1.0  6.0  4.0  4.0  47.0  1.0  1.0  3.0  1.0  3.0 ...       1.0     0   
3  1.0  6.0  4.0  4.0  47.0  1.0  1.0  3.0  1.0  3.0 ...       1.0     0   
4  1.0  6.0  4.0  4.0  47.0  1.0  1.0  3.0  1.0  3.0 ...       1.0     0   

   work  written  www.coxcsrreport.com.  year  yes  yield  you    ’  
0   1.0      1.0                    1.0   6.0  1.0    1.0  1.0  1.0  
1   1.0      1.0                    1.0   6.0  1.0    1.0  1.0  1.0  
2   1.0      1.0                    1.0   6.0  1.0    1.0  1.0  1.0  
3   1.0      1.0                    1.0   6.0  1.0    1.0  1.0  1.0  
4   1.0      1.0                    1.0   6.0  1.0    1.0  1.0  1.0  

[5 rows x 311 columns]
     %    '    (    )     ,    -    .    2    3    5 ...   without  word  \
0  1.0  6.0  4.0  4.0  

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A c

#### Below are the 10 ten terms with the strongest coefficients after TF-IDF transformation of word counts

In [96]:

logreg_dict_words

{'toktok:porter': ['will',
  'without',
  'word',
  'work',
  'www.coxcsrreport.com.',
  'year',
  'yes',
  'yield',
  'count',
  'with'],
 'toktok:stemmer': ['will',
  'without',
  'word',
  'work',
  'www.coxcsrreport.com.',
  'year',
  'yes',
  'yield',
  'count',
  'with'],
 'tree:porter': ['will',
  'without',
  'word',
  'work',
  'www.coxcsrreport.com.',
  'year',
  'yes',
  'yield',
  'count',
  'with'],
 'tree:stemmer': ['will',
  'without',
  'word',
  'work',
  'www.coxcsrreport.com.',
  'year',
  'yes',
  'yield',
  'count',
  'with']}

There is a marked difference after applying TF-IDF. Common words like we, an, and the semicolon are present when using simple word counts which are not present in TF-IDF. Hence there is no need to specify a set of stop-words, TF-IDF takes care of this by giving low weight to words which occur in all documents. 

Without TF_IDF: 'divers','scalabl','We','an','better','workstreams.','employ','team.',';','solut'
With TF-IDF: 'will','without', 'word','work', 'www.coxcsrreport.com.', 'year', 'yes','yield','count','with'